In [2]:
import sys, os
import numpy as np
np.set_printoptions(threshold=np.inf)
import pandas as pd
from utils.utils import read_txt, read_txt_np_int
from functools import lru_cache
from math import ceil, floor
from copy import deepcopy

# INPUT

In [30]:
inputfilename = './inputs/day18A.txt'

inputdata = read_txt(inputfilename)

testdata_small = ["5,4",
"4,2",
"4,5",
"3,0",
"2,1",
"6,3",
"2,4",
"1,5",
"0,6",
"3,3",
"2,6",
"5,1",
"1,2",
"5,5",
"2,5",
"6,5",
"1,4",
"0,4",
"6,4",
"1,1",
"6,1",
"1,0",
"0,5",
"1,6",
"2,0"]

mapsize_inputdata = (71,71)
mapsize_small = (7,7)

In [51]:
data_to_use, mapsize, bytes_to_fall = testdata_small, mapsize_small, 12
data_to_use, mapsize, bytes_to_fall = inputdata, mapsize_inputdata, 1024

starting_point = (0,0)
ending_point = (mapsize[0]-1,mapsize[1]-1)

computer_map = np.zeros(mapsize,dtype=int)
computer_map

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0,

## PART 1

In [ ]:
# Let the bytes fall
for byte in data_to_use[0:bytes_to_fall]:
    computer_map[int(byte.split(',')[1]),int(byte.split(',')[0])] = 1
    
computer_map

array([[0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 0, 0, 0]])

In [33]:
def find_shortest_path(computer_map, starting_position, final_position):

    current_position = starting_position
    path_guide_distances = {}
    path_guide_distances[current_position] = 0
    positions_to_check = set([current_position])

    while len(positions_to_check) > 0:
        current_position = positions_to_check.pop()
        current_distance = path_guide_distances[current_position]
        for new_position in [(current_position[0]+1,current_position[1]),
                             (current_position[0]-1,current_position[1]),
                             (current_position[0],current_position[1]+1),
                             (current_position[0],current_position[1]-1)]:
            
            # Out of bounds -- cannot continue
            if new_position[0] < 0 or new_position[0] >= computer_map.shape[0] or new_position[1] < 0 or new_position[1] >= computer_map.shape[1]:
                continue
            
            # Non valid position -- cannot continue
            if computer_map[new_position] > 0:
                continue

            # New position -- add to the list
            if new_position not in path_guide_distances:
                path_guide_distances[new_position] = current_distance + 1
                positions_to_check.add(new_position)

            # Already visited position -- update if shorter path found
            elif path_guide_distances[new_position] > current_distance + 1:
                path_guide_distances[new_position] = current_distance + 1
                positions_to_check.add(new_position)

    return path_guide_distances

In [34]:
path_map = find_shortest_path(computer_map, starting_point, ending_point)
print(path_map[ending_point])

312


## PART 2

In [47]:
def is_map_cuttoff(computer_map):
    # Get all corrupted positions in first row and last column
    size = computer_map.shape[0]
    corrupted_positions = set()
    for i in range(size):
        if computer_map[i,size-1] == 1:
            corrupted_positions.add((i,size-1))
        if computer_map[0,i] == 1:
            corrupted_positions.add((0,i))

    positions_to_explore = corrupted_positions.copy()
    # Now search for a path from these to the last
    while len(positions_to_explore) > 0:
        current_position = positions_to_explore.pop()
        for movement in [(1,0),(-1,0),(0,1),(0,-1),(1,1),(1,-1),(-1,1),(-1,-1)]:
            new_position = (current_position[0] + movement[0], current_position[1] + movement[1]) 

            # Out of bounds -- cannot continue
            if new_position[0] < 0 or new_position[0] >= computer_map.shape[0] or new_position[1] < 0 or new_position[1] >= computer_map.shape[1]:
                continue
            
            # Non corrupted position -- cannot continue
            if computer_map[new_position] == 0:
                continue

            # Otherwise add to the list IF it is new
            if new_position not in corrupted_positions:
                corrupted_positions.add(new_position)
                positions_to_explore.add(new_position)

    # Now check if we readhed the last row or first column
    for corrupted_position in corrupted_positions:
        if corrupted_position[0] == size-1 or corrupted_position[1] == 0:
            print(f"Cutof by reaching {corrupted_position}")
            return True
    
    return False

In [52]:
# Solve part 2
# Let the bytes fall one by one
bytecount = 0
for byte in data_to_use:
    bytecount += 1
    computer_map[int(byte.split(',')[1]),int(byte.split(',')[0])] = 1
    if is_map_cuttoff(computer_map):
        print(f"Cutoff reached after {byte} fell, for a total of {bytecount} bytes")
        break
    
computer_map

Cutof by reaching (70, 41)
Cutoff reached after 28,26 fell, for a total of 3038 bytes


array([[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
        0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 0, 1, 1],
       [0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0],
       [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
        1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0],
       [0, 0, 0,